In [18]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [80]:
def read_links(url, skip, append):
    mp = requests.get(url).content
    soup = BeautifulSoup(mp, 'html.parser')
    goodstart = 'https://www.nrcan.gc.ca/energy/fuel-prices/crude'
    all_links = list(set([lnk['href'] for lnk in soup.find_all('a', href=True) if lnk['href'].startswith(goodstart)]))
    for link in skip:
        if link in all_links:
            all_links.remove(link)
    all_links += append
    return all_links

In [85]:
dict_2015 = {
    'url': 'https://www.nrcan.gc.ca/energy/fuel-prices/crude/18122',
    'skip': [
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/17087', # monthly
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/16993', # 2014
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/18500', # 2013
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/18498', # 2012
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/18496', # 2011
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/18490', # 2010
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/18484', # 2009
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/18478', # 2008
        ],
    'append': ['https://www.nrcan.gc.ca/node/17963']
}
links_2015 = read_links(**dict_2015)
print(len(links_2015))

12


In [83]:

dict_2014 = {
    'url': 'https://www.nrcan.gc.ca/energy/fuel-prices/crude/16993',
    'skip': [
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/18500', # 2013
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/18498', # 2012
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/18496', # 2011
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/18490', # 2010
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/18484', # 2009
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/18478', # 2008
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/15085',
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/15087',
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/15894',
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/15707',
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/18535',
        'https://www.nrcan.gc.ca/energy/fuel-prices/crude/14456'
        ],
    'append': []
}
links_2014 = read_links(**dict_2014)
print(len(links_2014))

12


In [56]:
def page_to_df(link):
    """Read a page and return a dataframe"""
    df = pd.read_html(link)[0].set_index('Date').drop('Average')
    return df

In [53]:
link = 'https://www.nrcan.gc.ca/energy/fuel-prices/crude/21532'

In [54]:
df = pd.read_html(link)[0].set_index('Date').drop('Average')

In [100]:
df.tail()

,Exchange Rate,WTI Cushing,"Brent Sullom Voe, UK",Mixed Sweet Blend Edmonton,Synthetic Edmonton,Western Canada Select Hardisty,Implied Bitumen Hardisty
Date,,,,,,,
2018-10-27,-,-,-,-,-,-,-
2018-10-28,-,-,-,-,-,-,-
2018-10-29,1.3067,551,607,304,321,206,104
2018-10-30,1.2998,541,589,271,292,210,117
2018-10-31,1.2998,534,590,251,285,182,82


In [101]:
pd.read_html(links_main[-1],header=0)[0].tail()

,Date,Exchange Rate,Cdn Light Sweet Edmonton,Western Canada Select Hardisty,Cdn Light Sweet Chicago,WTI Chicago,Western Canada Select Chicago,Brent Montreal
26,2016-11-27,-,-,-,-,-,-,-
27,2016-11-28,13401,358,264,396,413,308,425
28,2016-11-29,1.3435,345,250,383,398,293,411
29,2016-11-30,13426,378,286,417,433,329,445
30,Average,13432,352,258,390,402,302,412


In [114]:
def normalize_cols(cols):
    return ['_'.join(col.split()).lower() for col in cols]

def maindf():
    main_dict = {
        'url': 'https://www.nrcan.gc.ca/energy/oil-sands/18087',
        'skip': [
            'https://www.nrcan.gc.ca/energy/fuel-prices/crude/19336',
            'https://www.nrcan.gc.ca/energy/fuel-prices/crude/17087',
            'https://www.nrcan.gc.ca/energy/fuel-prices/crude/18122',
            'https://www.nrcan.gc.ca/energy/fuel-prices/crude/16993'
            ],
        'append': ['https://www.nrcan.gc.ca/energie/prix-carburant/brut/19204']
    }
    links_main = read_links(**main_dict)
    dflist = []
    for link in links_main:
        df_part = pd.read_html(link, header=0)[0].set_index('Date').drop('Average')
        df_part.columns = normalize_cols(df_part.columns)
        dflist.append(df_part)
    df = pd.concat([dfp for dfp in dflist], sort=True)
    return df.sort_index()
df_main = maindf()

In [118]:
df_main.sort_index().tail()

,brent_chicago,brent_montreal,brent_sarnia,"brent_sullom_voe,_uk",cdn_heavy_chicago,cdn_heavy_hardisty,cdn_light_chicago,cdn_light_sweet,cdn_light_sweet*,cdn_light_sweet_chicago,cdn_light_sweet_edmonton,exchange_rate,implied_bitumen_hardisty,mixed_sweet_blend_edmonton,synthetic_edmonton,western_canada_select_chicago,western_canada_select_hardisty,wti_chicago,wti_cushing,wti_nymex_chicago
Date,,,,,,,,,,,,,,,,,,,,
2018-10-28,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-,-,-,NaN,-,NaN,-,NaN
2018-10-29,NaN,NaN,NaN,607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3067,104,304,321,NaN,206,NaN,551,NaN
2018-10-30,NaN,NaN,NaN,589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2998,117,271,292,NaN,210,NaN,541,NaN
2018-10-31,NaN,NaN,NaN,590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2998,82,251,285,NaN,182,NaN,534,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,---------------------------------- $ Cdn/m3 --...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
